
# 🧰 Paso previo: Descomprimir MetStation y reunir archivos `.25O` por estación


In [1]:

# === CONFIGURACIÓN DE RUTAS (AJUSTA SOLO SI ES NECESARIO) ===
from pathlib import Path

# Carpeta con los .zip (según lo solicitado)
ZIP_ROOT = Path(r"C:\Users\varga\Box\Data\RinexZipFiles\MetStation\Last")

# Carpeta base donde se crearán subcarpetas por estación (nombre del zip sin extensión)
DEST_ROOT = Path(r"C:\Users\varga\Box\Data\nc Files\MetSta")

# Subcadena para identificar la carpeta RINEX (como en tu flujo original)
RINEX_SUBSTR = "RINEX_3_03"

# Patrón de zip a considerar
ZIP_PATTERN = "MetStation*.zip"

# Carpeta donde se extraerán los zips (se crea junto a ZIP_ROOT)
UNZIP_ROOT = ZIP_ROOT / "unzipped"

# Política de sobre-escritura de .25O
OVERWRITE_25O = False

ZIP_ROOT, DEST_ROOT, UNZIP_ROOT

(WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/MetStation/Last'),
 WindowsPath('C:/Users/varga/Box/Data/nc Files/MetSta'),
 WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/MetStation/Last/unzipped'))

In [2]:

# === LÓGICA: extraer zips y copiar TODOS los *.25O a una sola carpeta ===
import zipfile, shutil

def safe_extract(zf: zipfile.ZipFile, dest: Path):
    for member in zf.infolist():
        member_path = dest / member.filename
        if not str(member_path.resolve()).startswith(str(dest.resolve())):
            raise RuntimeError(f"Entrada peligrosa en ZIP: {member.filename}")
        zf.extract(member, dest)

def extract_all_zips(root: Path, unzip_root: Path, pattern: str):
    unzip_root.mkdir(parents=True, exist_ok=True)
    zips = sorted(root.glob(pattern), key=lambda p: p.name.lower())
    extracted = []
    for z in zips:
        target_dir = unzip_root / z.stem
        if target_dir.exists():
            extracted.append(target_dir)
            continue
        try:
            with zipfile.ZipFile(z) as zf:
                print(f"[EXTRACT] {z.name} -> {target_dir.name}")
                target_dir.mkdir(parents=True, exist_ok=True)
                safe_extract(zf, target_dir)
                extracted.append(target_dir)
        except Exception as e:
            print(f"[ERROR] {z.name}: {e}")
    return extracted

def find_rinex_dirs(base: Path, substr: str):
    sub = substr.lower()
    return [p for p in base.rglob("*") if p.is_dir() and sub in p.name.lower()]

def copy_25O_flat(unzip_root: Path, dest_root: Path, rinex_substr: str, overwrite=False):
    dest_root.mkdir(parents=True, exist_ok=True)
    total = 0
    for station_dir in sorted(p for p in unzip_root.iterdir() if p.is_dir()):
        rinex_dirs = find_rinex_dirs(station_dir, rinex_substr)
        if rinex_dirs:
            for rdir in rinex_dirs:
                for f in rdir.glob("*.25O"):
                    df = dest_root / f.name
                    if df.exists() and not overwrite:
                        continue
                    shutil.copy2(f, df)
                    total += 1
        else:
            for f in station_dir.rglob("*.25O"):
                df = dest_root / f.name
                if df.exists() and not overwrite:
                    continue
                shutil.copy2(f, df)
                total += 1
    return total

# === Ejecutar flujo ===
DEST_ROOT.mkdir(parents=True, exist_ok=True)
extracted_dirs = extract_all_zips(ZIP_ROOT, UNZIP_ROOT, ZIP_PATTERN)
for p in sorted(UNZIP_ROOT.iterdir() if UNZIP_ROOT.exists() else []):
    if p.is_dir() and p not in extracted_dirs:
        extracted_dirs.append(p)

total_copiados = copy_25O_flat(UNZIP_ROOT, DEST_ROOT, RINEX_SUBSTR, overwrite=OVERWRITE_25O)
print("\n===== RESUMEN (Notebook) =====")
print(f"Zips procesados/existentes: {len(extracted_dirs)}")
print(f"Archivos *.25O copiados en total: {total_copiados}")
print(f"Destino único: {DEST_ROOT}")


[EXTRACT] MetStation(1).zip -> MetStation(1)
[EXTRACT] MetStation(10).zip -> MetStation(10)
[EXTRACT] MetStation(100).zip -> MetStation(100)
[EXTRACT] MetStation(101).zip -> MetStation(101)
[EXTRACT] MetStation(102).zip -> MetStation(102)
[EXTRACT] MetStation(103).zip -> MetStation(103)
[EXTRACT] MetStation(104).zip -> MetStation(104)
[EXTRACT] MetStation(105).zip -> MetStation(105)
[EXTRACT] MetStation(106).zip -> MetStation(106)
[EXTRACT] MetStation(107).zip -> MetStation(107)
[EXTRACT] MetStation(108).zip -> MetStation(108)
[EXTRACT] MetStation(109).zip -> MetStation(109)
[EXTRACT] MetStation(11).zip -> MetStation(11)
[EXTRACT] MetStation(110).zip -> MetStation(110)
[EXTRACT] MetStation(111).zip -> MetStation(111)
[EXTRACT] MetStation(112).zip -> MetStation(112)
[EXTRACT] MetStation(113).zip -> MetStation(113)
[EXTRACT] MetStation(114).zip -> MetStation(114)
[EXTRACT] MetStation(115).zip -> MetStation(115)
[EXTRACT] MetStation(116).zip -> MetStation(116)
[EXTRACT] MetStation(117).zi

# Processing of Files for the MANCHI Station

Test notebook for data processing

## Libraries

In [6]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime 
import shutil
from pathlib import Path


## Presets

### Time intervals

In [6]:
# Define the start day
startday = pd.to_datetime('25-04-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-04-25 00:00:00, 2025-04-26 00:00:00),
               [2025-04-26 00:00:00, 2025-04-27 00:00:00),
               [2025-04-27 00:00:00, 2025-04-28 00:00:00),
               [2025-04-28 00:00:00, 2025-04-29 00:00:00),
               [2025-04-29 00:00:00, 2025-04-30 00:00:00),
               [2025-04-30 00:00:00, 2025-05-01 00:00:00),
               [2025-05-01 00:00:00, 2025-05-02 00:00:00),
               [2025-05-02 00:00:00, 2025-05-03 00:00:00),
               [2025-05-03 00:00:00, 2025-05-04 00:00:00),
               [2025-05-04 00:00:00, 2025-05-05 00:00:00)],
              dtype='interval[datetime64[ns], left]')

## Process Rinex Files

In [7]:
from pathlib import Path

# Carpeta única donde se copiaron todos los .25O
root = Path(r"C:\Users\varga\Box\Data\nc Files\MetSta")

# Buscar todos los archivos con extensión .25O
files = list(root.glob("*.25O"))

print("Found", len(files), "files:")
for f in files:
    print(f.name)


Found 133 files:
MetStation_raw_20250413000420.25O
MetStation_raw_20250414000421.25O
MetStation_raw_20250415000422.25O
MetStation_raw_20250415221018.25O
MetStation_raw_20250416221018.25O
MetStation_raw_20250417221019.25O
MetStation_raw_20250418221020.25O
MetStation_raw_20250419221021.25O
MetStation_raw_20250420221022.25O
MetStation_raw_20250421221023.25O
MetStation_raw_20250422221025.25O
MetStation_raw_20250423221026.25O
MetStation_raw_20250424221027.25O
MetStation_raw_20250425221028.25O
MetStation_raw_20250426221029.25O
MetStation_raw_20250427221030.25O
MetStation_raw_20250430150820.25O
MetStation_raw_20250501150821.25O
MetStation_raw_20250502150821.25O
MetStation_raw_20250503150822.25O
MetStation_raw_20250504150823.25O
MetStation_raw_20250505150824.25O
MetStation_raw_20250506150825.25O
MetStation_raw_20250507150825.25O
MetStation_raw_20250508150826.25O
MetStation_raw_20250509150827.25O
MetStation_raw_20250510150827.25O
MetStation_raw_20250511150828.25O
MetStation_raw_20250512150829.2

In [7]:
# Carpeta donde dejaste TODOS los .25O
DEST = Path(r"C:\Users\varga\Box\Data\nc Files\MetSta")

# 1) Archivos de entrada (RINEX .25O) - genérico
pattern = {
    'MetSta': str(DEST / '*.25O')
    # Si eventualmente hay subcarpetas dentro de MetSta, usá:
    # 'MetSta': str(DEST / '**' / '*.25O')
    # (y agregá recursive=True en glob si lo usás manualmente)
}

# 2) Carpeta de salida para NetCDF (dentro de MetSta)
outdir = DEST / "processed"
outdir.mkdir(parents=True, exist_ok=True)
outputdir = {'MetSta': str(outdir)}

# (Opcional) chequeo rápido
files = glob.glob(pattern['MetSta'])
print(f'Found {len(files)} .25O files (showing up to 5):', files[:5])
if len(files) == 0:
    raise FileNotFoundError("No .25O files found in DEST folder. Check DEST and file existence.")

# 3) Variables a conservar (SNR)
keepvars = ['S?', 'S??']

# 4) Preprocesamiento
result = gv.preprocess(
    pattern,
    interval='15s',
    keepvars=keepvars,
    outputdir=outputdir,
    outputresult=True
)

# 5) Extraer con seguridad el primer dataset procesado (xarray.Dataset)
metsta_res = result.get('MetSta', [])
if isinstance(metsta_res, list):
    if not metsta_res:
        raise ValueError("No datasets produced for 'MetSta'. Check the pattern and/or keepvars.")
    obs = metsta_res[0]
elif isinstance(metsta_res, xr.Dataset):
    obs = metsta_res
else:
    raise TypeError(f"Unexpected return type for 'MetSta': {type(metsta_res)}")

obs


Found 133 .25O files (showing up to 5): ['C:\\Users\\varga\\Box\\Data\\nc Files\\MetSta\\MetStation_raw_20250413000420.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MetSta\\MetStation_raw_20250414000421.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MetSta\\MetStation_raw_20250415000422.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MetSta\\MetStation_raw_20250415221018.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MetSta\\MetStation_raw_20250416221018.25O']
Created a temporary directory at C:\Users\varga\AppData\Local\Temp\tmp8_tc12px
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\MetSta\processed*.nc
C:\Users\varga\Box\Data\nc Files\MetSta\MetStation_raw_20250413000420.25O exists | Reading...
Processed 88072 out of 3195452 lines (2.8%)
Processed 865092 out of 3195452 lines (27.1%)
Processed 1597967 out of 3195452 lines (50.0%)
Processed 2206545 out of 3195452 lines (69.1%)
Processed 2657434 out of 3195452 lines (83.2%)
Observation file  C:\Users\varga\Bo

GFZ0MGXRAP_20251020000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 315kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2361//GFZ0MGXRAP_20251020000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251020000_01D_05M_ORB.SP3 file is read in 10.61 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz: 1.03MB [00:03, 327kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3 file is read in 9.43 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz

...

GFZ0MGXRAP_20251090000_01D_30S_CLK.CLK.gz: 4.80MB [00:31, 159kB/s]                             


 | Download completed for GFZ0MGXRAP_20251090000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251090000_01D_30S_CLK.CLK file is read in 1.87 seconds
SP3 interpolation is done in 3.83 seconds
Saved 103192 individual observations in C:\Users\varga\Box\Data\nc Files\MetSta\processed\MetStation_raw_20250417221019.nc
C:\Users\varga\Box\Data\nc Files\MetSta\MetStation_raw_20250418221020.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\MetSta\MetStation_raw_20250418221020.25O  is read in 2.35 seconds.
Processing 102383 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251080000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251080000_01D_05M_ORB.SP3 file is read in 0.53 seconds
C:\Users\varga\AppData\Local\Temp\tmp8_tc12px\GFZ0MGXRAP_20251090000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\AppData\Local\Temp\tmp8_tc

GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz: 0.00B [33:51, ?B/s]


<urlopen error [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto>
 | Requested file ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz cannot be not found!


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\varga\\AppData\\Local\\Temp\\tmp8_tc12px\\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3'